In [48]:
from datetime import datetime
import glob
import time
import numpy as np
import talib

# from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

import plotly as py
import cufflinks as cf
import pandas as pd
import plotly.tools as tls
import plotly.graph_objs as go

import plotly.figure_factory as ff
init_notebook_mode(connected=True)

import pyodbc
from sqlalchemy import create_engine

#  todo driver 修正
def conn():
    return pyodbc.connect(
    'DRIVER={ODBC Driver 13 for SQL Server};' +
    'SERVER=localhost;' + 'PORT=1443;' +
    'DATABASE=StockDB;' +
    'UID=sa;' + 'PWD=admin;' )

# Create the sqlalchemy engine using the pyodbc connection
engine = create_engine( "mssql+pyodbc://?driver=ODBC+Driver+13?charset=utf8", creator = conn )
con = engine.connect( )
con.execute( "SET LANGUAGE us_english; set dateformat ymd;" )
con.close()

In [2]:
query_num = '2362'

In [24]:
cmd = """SELECT
      [date],
      [open_price],
      [high_price],
      [low_price],
      [close_price],
      [volume],
      [ma3],
      [ma10], 
      [ma60],
      [ma120],
      [ma240] FROM [dbo].[TECH_D] WHERE stock = \'{}\' ORDER BY date DESC"""

tech_d_df = pd.read_sql_query(cmd.format(query_num), engine)

# tech_d_df.head( )

tech_d_df['date'] = pd.to_datetime(tech_d_df['date'])

mask = (tech_d_df['date'] > '2017-1-1')

tech_d_df = tech_d_df.loc[mask]

trace0 = go.Candlestick( x   = tech_d_df[ 'date' ],
                       open  = tech_d_df[ 'open_price' ],
                       high  = tech_d_df[ 'high_price' ],
                       low   = tech_d_df[ 'low_price' ],
                       close = tech_d_df[ 'close_price' ],
                       name = 'K線',
                       line = dict( width = 1 ) )

trace1 = go.Scatter(
    x = tech_d_df[ 'date' ],
    y = tech_d_df[ 'ma3' ],
    mode = 'lines',
    name = '3日均線',
    line = dict( width = 1 )
)

trace2 = go.Scatter(
    x = tech_d_df[ 'date' ],
    y = tech_d_df[ 'ma10' ],
    mode = 'lines',
    name = '10日均線',
    line = dict( width = 1 )
)

trace3 = go.Scatter(
    x = tech_d_df[ 'date' ],
    y = tech_d_df[ 'ma60' ],
    mode = 'lines',
    name = '60日均線',
    line = dict( width = 1, color='rgb(82, 82, 82)' )
)

trace4 = go.Scatter(
    x = tech_d_df[ 'date' ],
    y = tech_d_df[ 'ma120' ],
    mode = 'lines',
    name = '120日均線',
    line = dict( width = 1 )
)

trace5 = go.Scatter(
    x = tech_d_df[ 'date' ],
    y = tech_d_df[ 'ma240' ],
    mode = 'lines',
    name = '240日均線',
    line = dict( width = 1 )
)

data = [ trace0, trace1, trace2, trace3, trace4, trace5 ]



py.offline.iplot( data, filename='ohlc_datetime')

# layout = {
#     'title': query_num,
#     'yaxis': { 'title': '股價' } }
# fig = dict( data = [trace], layout = layout )
# py.offline.iplot( fig, filename='ohlc_datetime')

# tech_d_df.head( )

In [44]:
C = np.array( tech_d_df[ 'volume' ], dtype = float, ndmin = 1 )

for val in [ 5, 10, 20 ]:
    tech_d_df[ 'MA' + str( val ) ] = talib.SMA( C, val )

trace1 = go.Bar(
    
    x = tech_d_df[ 'date' ],
    y = tech_d_df[ 'volume' ],
    name = '成交量',
    
    marker=dict(
        color='rgb(158,202,225)',
        line=dict(
            color='rgb(8,48,107)',
            width=1.5,
        )
    ),
    
    opacity=0.6
)

trace2 = go.Scatter(
    x = tech_d_df[ 'date' ],
    y = tech_d_df[ 'MA5' ],
    mode = 'lines',
    name = '成交均線5',
    line = dict( width = 1 )
)

trace3 = go.Scatter(
    x = tech_d_df[ 'date' ],
    y = tech_d_df[ 'MA10' ],
    mode = 'lines',
    name = '成交均線10',
    line = dict( width = 1 )
)

trace4 = go.Scatter(
    x = tech_d_df[ 'date' ],
    y = tech_d_df[ 'MA20' ],
    mode = 'lines',
    name = '成交均線20',
    line = dict( width = 1 )
)

data = [ trace1, trace2, trace3, trace4 ]

py.offline.iplot( data, filename='ohlc_datetime')

In [58]:
# 外資
# 投信
# 自營商

cmd = """SELECT [stock]
      ,[date]
      ,[foreign_sell]
      ,[investment_sell]
      ,[dealer_sell]
      ,[single_day_sell]
      ,[foreign_estimate]
      ,[investment_estimate]
      ,[dealer_estimate]
      ,[single_day_estimate]
      ,[foreign_ratio]
      ,[investment_ratio]
  FROM [dbo].[INVESTORS]
  WHERE stock = \'{}\' ORDER BY date DESC"""
  
investors_df = pd.read_sql_query( cmd.format( query_num ), engine )


trace1 = go.Bar(
    
    x = investors_df[ 'date' ],
    y = investors_df[ 'foreign_sell' ],
    
    name = '外資',
    
    marker=dict(
        color='rgb(158,202,225)',
        line=dict(
            color='rgb(8,48,107)',
            width=1.5,
        )
    ),
    
    opacity = 0.6
)

trace2 = go.Scatter(
    x = investors_df[ 'date' ],
    y = investors_df[ 'foreign_estimate' ],
    mode = 'lines',
    name = '外資累計',
    line = dict( width = 1 ),
    yaxis='y2'
)

data = [ trace1, trace2 ]

layout = go.Layout(
    
    title='外資買賣超',
    
    yaxis=dict(
        title='單日買賣量'
    ),
    
    yaxis2 = dict(
        title='累計買賣量',
        titlefont=dict(
            color='rgb(148, 103, 189)'
        ),
        tickfont=dict(
            color='rgb(148, 103, 189)'
        ),
        overlaying='y',
        side='right'
    )
)

fig = go.Figure(data=data, layout=layout)
py.offline.iplot( fig, filename='ohlc_datetime' )

# table = ff.create_table( investors_df.head( ) )
# py.offline.iplot( table, filename='jupyter-table1' )
# investors_df.head( )

In [59]:
trace1 = go.Bar(
    
    x = investors_df[ 'date' ],
    y = investors_df[ 'investment_sell' ],
    
    name = '投信',
    
    marker=dict(
        color='rgb(158,202,225)',
        line=dict(
            color='rgb(8,48,107)',
            width=1.5,
        )
    ),
    
    opacity = 0.6
)

trace2 = go.Scatter(
    x = investors_df[ 'date' ],
    y = investors_df[ 'investment_estimate' ],
    mode = 'lines',
    name = '投信累計',
    line = dict( width = 1 ),
    yaxis='y2'
)

data = [ trace1, trace2 ]

layout = go.Layout(
    
    title='投信買賣超',
    
    yaxis=dict(
        title='單日買賣量'
    ),
    
    yaxis2 = dict(
        title='累計買賣量',
        titlefont=dict(
            color='rgb(148, 103, 189)'
        ),
        tickfont=dict(
            color='rgb(148, 103, 189)'
        ),
        overlaying='y',
        side='right'
    )
)

fig = go.Figure(data=data, layout=layout)
py.offline.iplot( fig, filename='ohlc_datetime' )

In [60]:
trace1 = go.Bar(
    
    x = investors_df[ 'date' ],
    y = investors_df[ 'dealer_sell' ],
    
    name = '自營商',
    
    marker=dict(
        color='rgb(158,202,225)',
        line=dict(
            color='rgb(8,48,107)',
            width=1.5,
        )
    ),
    
    opacity = 0.6
)

trace2 = go.Scatter(
    x = investors_df[ 'date' ],
    y = investors_df[ 'dealer_estimate' ],
    mode = 'lines',
    name = '自營商累計',
    line = dict( width = 1 ),
    yaxis='y2'
)

data = [ trace1, trace2 ]

layout = go.Layout(
    
    title='自營商買賣超',
    
    yaxis=dict(
        title='單日買賣量'
    ),
    
    yaxis2 = dict(
        title='累計買賣量',
        titlefont=dict(
            color='rgb(148, 103, 189)'
        ),
        tickfont=dict(
            color='rgb(148, 103, 189)'
        ),
        overlaying='y',
        side='right'
    )
)

fig = go.Figure(data=data, layout=layout)
py.offline.iplot( fig, filename='ohlc_datetime' )

In [64]:
cmd = """SELECT [stock]
      ,[date]
      ,[Financing_Buy]
      ,[Financing_Sell]
      ,[Financing_PayOff]
      ,[Financing_Over]
      ,[Financing_Increase]
      ,[Financing_Limit]
      ,[Financing_Use]
      ,[Margin_Sell]
      ,[Margin_Buy]
      ,[Margin_PayOff]
      ,[Margin_Over]
      ,[Margin_Increase]
      ,[Margin_Ratio]
      ,[Margin_Offset]
  FROM [dbo].[MARGIN]
  WHERE stock = \'{}\' ORDER BY date DESC"""
  
margin_df = pd.read_sql_query( cmd.format( query_num ), engine )

trace1 = go.Bar(
    
    x = margin_df[ 'date' ],
    y = margin_df[ 'Financing_Buy' ],
    
    name = '融資買進',
    
    marker=dict(
        color='rgb(158,202,225)',
        line=dict(
            color='rgb(8,48,107)',
            width=1.5,
        )
    ),
    
    opacity = 0.6
)

data = [ trace1 ]

layout = go.Layout(
    
    title='融資買進',
    
    yaxis=dict(
        title='單日買量'
    ),
    
    yaxis2 = dict(
        title='累計買賣量',
        titlefont=dict(
            color='rgb(148, 103, 189)'
        ),
        tickfont=dict(
            color='rgb(148, 103, 189)'
        ),
        overlaying='y',
        side='right'
    )
)

fig = go.Figure(data=data, layout=layout)
py.offline.iplot( fig, filename='ohlc_datetime' )

# margin_df.head()

In [65]:
trace1 = go.Bar(
    
    x = margin_df[ 'date' ],
    y = margin_df[ 'Financing_Sell' ],
    
    name = '融資賣出',
    
    marker=dict(
        color='rgb(158,202,225)',
        line=dict(
            color='rgb(8,48,107)',
            width=1.5,
        )
    ),
    
    opacity = 0.6
)

data = [ trace1 ]

layout = go.Layout(
    
    title='融資賣出',
    
    yaxis=dict(
        title='單日賣量'
    ),
    
    yaxis2 = dict(
        title='累計買賣量',
        titlefont=dict(
            color='rgb(148, 103, 189)'
        ),
        tickfont=dict(
            color='rgb(148, 103, 189)'
        ),
        overlaying='y',
        side='right'
    )
)

fig = go.Figure(data=data, layout=layout)
py.offline.iplot( fig, filename='ohlc_datetime' )

# margin_df.head()

In [7]:
cmd = """SELECT [stock]
      ,[date]
      ,[lend_over]
      ,[lend_chang]
      ,[lend_return]
      ,[lend_diff]
      ,[sell_over]
      ,[sell_day]
      ,[sell_day_return]
      ,[sell_day_diff]
      ,[sell_day_limit]
  FROM [dbo].[LEND]
  WHERE stock = \'{}\' ORDER BY date DESC"""
  
lend_df = pd.read_sql_query( cmd.format( query_num ), engine )

lend_df.head()

,stock,date,lend_over,lend_chang,lend_return,lend_diff,sell_over,sell_day,sell_day_return,sell_day_diff,sell_day_limit
0,2362,2018-03-16,2792000,0,0,0,1557000,0,0,0,170597
1,2362,2018-03-15,2792000,0,34000,-34000,1557000,7000,34000,-27000,168381
2,2362,2018-03-14,2826000,0,0,0,1584000,0,0,0,166674
3,2362,2018-03-13,2826000,0,0,0,1584000,7000,0,7000,165115
4,2362,2018-03-12,2826000,0,1000,-1000,1577000,1000,0,1000,163894


In [9]:
# u'當月營收', 
# u'上月營收', 
# u'去年當月營收', 
# u'上月比較增減(%)', 
# u'去年同月增減(%)', 
# u'當月累計營收', 
# u'去年累計營收', 
# u'前期比較增減(%)' 

cmd = """SELECT [stock]
      ,[date]
      ,[Month_Revenue]
      ,[Last_Month_Revenue]
      ,[Last_Year_Revenue]
      ,[Last_Month_Ratio]
      ,[Last_Year_Ration]
      ,[Month_Acc_Revenue]
      ,[Last_Year_Acc_Revenue]
      ,[ration]  
  FROM [dbo].[REVENUE]
  WHERE stock = \'{}\' ORDER BY date DESC"""
  
revenue_df = pd.read_sql_query( cmd.format( query_num ), engine )

# revenue_df.head()

In [10]:
fig = tls.make_subplots( rows=8, cols=1, shared_xaxes=False )
#                         subplot_titles=( '集中度', 'K線', '主力買賣超', '主力買賣超') )

fig.append_trace( {'x': revenue_df['date'], 
                   'y': revenue_df['Month_Revenue'], 
                   'type': 'scatter', 
                   'name': '當月營收' }, 1, 1 )

fig.append_trace( {'x': revenue_df['date'], 
                   'y': revenue_df['Last_Month_Revenue'], 
                   'type': 'scatter', 
                   'name': '上月營收' }, 2, 1 )

fig.append_trace( {'x': revenue_df['date'], 
                   'y': revenue_df['Last_Year_Revenue'], 
                   'type': 'scatter', 
                   'name': '去年當月營收' }, 3, 1 )

fig.append_trace( {'x': revenue_df['date'], 
                   'y': revenue_df['Last_Month_Ratio'], 
                   'type': 'scatter', 
                   'name': '上月比較增減%' }, 4, 1 )

fig.append_trace( {'x': revenue_df['date'], 
                   'y': revenue_df['Last_Year_Ration'], 
                   'type': 'scatter', 
                   'name': '去年同月增減%' }, 5, 1 )

fig.append_trace( {'x': revenue_df['date'], 
                   'y': revenue_df['Month_Acc_Revenue'], 
                   'type': 'scatter', 
                   'name': '當月累計營收' }, 6, 1 )

fig.append_trace( {'x': revenue_df['date'], 
                   'y': revenue_df['Last_Year_Acc_Revenue'], 
                   'type': 'scatter', 
                   'name': '去年累計營收' }, 7, 1 )

fig.append_trace( {'x': revenue_df['date'], 
                   'y': revenue_df['ration'], 
                   'type': 'scatter', 
                   'name': '前期比較增減' }, 8, 1 )

py.offline.iplot( fig, filename= '月營收' )

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x2,y2 ]
[ (3,1) x3,y3 ]
[ (4,1) x4,y4 ]
[ (5,1) x5,y5 ]
[ (6,1) x6,y6 ]
[ (7,1) x7,y7 ]
[ (8,1) x8,y8 ]



In [11]:
cmd = """SELECT    
      [date],
      [open_price],
      [high_price],
      [low_price],
      [close_price],
      [volume] FROM [dbo].[TECH_D] WHERE stock = \'{}\' ORDER BY date DESC"""

tech_d_df = pd.read_sql_query( cmd.format( query_num ), engine )

# tech_d_df.head( )

tech_d_df[ 'date' ] = pd.to_datetime( tech_d_df[ 'date' ] )  

mask = ( tech_d_df['date'] > '2018-1-1' ) 

tech_d_df = tech_d_df.loc[ mask ]

trace = go.Candlestick( x    = tech_d_df[ 'date' ],
                       open  = tech_d_df[ 'open_price' ],
                       high  = tech_d_df[ 'high_price' ],
                       low   = tech_d_df[ 'low_price' ],
                       close = tech_d_df[ 'close_price' ] )
layout = {
    'title': '大成鋼',
    'yaxis': { 'title': '股價' } }

fig = dict( data = [trace], layout = layout )
                       
py.offline.iplot( fig, filename='candlestick_datetime' )

In [12]:
path = r'C:\workspace\stock\01_Day process\券商分點\籌碼集中暫存.csv'

csv_df = pd.read_csv(
    path,
    sep=',',
    encoding='utf8',
    false_values='NA',
    dtype={
        '股號': str,
        '日期': str
    },
)

filename = '{}_籌碼集中.html'.format(query_num)

csv_df = csv_df[csv_df['股號'] == query_num]
csv_df = csv_df[::-1]

# hover_text = df.apply( lambda r: '<br>'.join( [ '{}: {}'.format( c, r[c] )
#                                             for c in df.columns ] ), axis=1 )

fig = tls.make_subplots(rows=3, cols=2, shared_xaxes=True)
#                         subplot_titles=( '集中度', 'K線', '主力買賣超', '主力買賣超') )

fig.append_trace({
    'x': csv_df['日期'],
    'y': csv_df['01天集中%'],
    'type': 'scatter',
    'name': '01天集中%'
}, 1, 2)

fig.append_trace({
    'x': csv_df['日期'],
    'y': csv_df['05天集中%'],
    'type': 'scatter',
    'name': '05天集中%'
}, 1, 2)

fig.append_trace({
    'x': csv_df['日期'],
    'y': csv_df['收盤'],
    'type': 'scatter',
    'name': '收盤'
}, 2, 2)

fig.append_trace({
    'x': csv_df['日期'],
    'y': csv_df['01天主力買賣超(張)'],
    'type': 'bar',
    'name': '張'
}, 3, 2)

trace = go.Candlestick(
    x=tech_d_df['date'],
    open=tech_d_df['open_price'],
    high=tech_d_df['high_price'],
    low=tech_d_df['low_price'],
    close=tech_d_df['close_price'])

fig.append_trace(trace, 1, 1)

py.offline.iplot(fig, filename=filename)

This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y2 ]
[ (2,1) x1,y3 ]  [ (2,2) x2,y4 ]
[ (3,1) x1,y5 ]  [ (3,2) x2,y6 ]



In [13]:
cmd = """SELECT [stock]
      ,[date]
      ,[Share_Rating_People_1_999]
      ,[Share_Rating_Unit_1_999]
      ,[Share_Rating_Proportion_1_999]
      ,[Share_Rating_People_1000_5000]
      ,[Share_Rating_Unit_1000_5000]
      ,[Share_Rating_Proportion_1000_5000]
      ,[Share_Rating_People_5001_10000]
      ,[Share_Rating_Unit_5001_10000]
      ,[Share_Rating_Proportion_5001_10000]
      ,[Share_Rating_People_10001_15000]
      ,[Share_Rating_Unit_10001_15000]
      ,[Share_Rating_Proportion_10001_15000]
      ,[Share_Rating_People_15001_20000]
      ,[Share_Rating_Unit_15001_20000]
      ,[Share_Rating_Proportion_15001_20000]
      ,[Share_Rating_People_20001_30000]
      ,[Share_Rating_Unit_20001_30000]
      ,[Share_Rating_Proportion_20001_30000]
      ,[Share_Rating_People_30001_40000]
      ,[Share_Rating_Unit_30001_40000]
      ,[Share_Rating_Proportion_30001_40000]
      ,[Share_Rating_People_40001_50000]
      ,[Share_Rating_Unit_40001_50000]
      ,[Share_Rating_Proportion_40001_50000]
      ,[Share_Rating_People_50001_100000]
      ,[Share_Rating_Unit_50001_100000]
      ,[Share_Rating_Proportion_50001_100000]
      ,[Share_Rating_People_100001_200000]
      ,[Share_Rating_Unit_100001_200000]
      ,[Share_Rating_Proportion_100001_200000]
      ,[Share_Rating_People_200001_400000]
      ,[Share_Rating_Unit_200001_400000]
      ,[Share_Rating_Proportion_200001_400000]
      ,[Share_Rating_People_400001_600000]
      ,[Share_Rating_Unit_400001_600000]
      ,[Share_Rating_Proportion_400001_600000]
      ,[Share_Rating_People_600001_800000]
      ,[Share_Rating_Unit_600001_800000]
      ,[Share_Rating_Proportion_600001_800000]
      ,[Share_Rating_People_800001_1000000]
      ,[Share_Rating_Unit_800001_1000000]
      ,[Share_Rating_Proportion_800001_1000000]
      ,[Share_Rating_People_Up_1000001]
      ,[Share_Rating_Unit_Up_1000001]
      ,[Share_Rating_Proportion_Up_1000001]
  FROM [dbo].[TDCC]
  WHERE stock = \'{}\' ORDER BY date DESC"""
  

tdcc_df = pd.read_sql_query( cmd.format( query_num ), engine )

tdcc_df.head()

tdcc_df[ '散戶持股比例100張以下' ] = tdcc_df[ 'Share_Rating_Proportion_1_999' ] + \
tdcc_df['Share_Rating_Proportion_1000_5000']  + \
tdcc_df['Share_Rating_Proportion_5001_10000'] + \
tdcc_df['Share_Rating_Proportion_10001_15000'] + \
tdcc_df['Share_Rating_Proportion_15001_20000'] +\
tdcc_df['Share_Rating_Proportion_20001_30000'] +\
tdcc_df['Share_Rating_Proportion_30001_40000'] +\
tdcc_df['Share_Rating_Proportion_40001_50000'] +\
tdcc_df['Share_Rating_Proportion_50001_100000'] 

tdcc_df['散戶持股比例200張以下'] = tdcc_df[ '散戶持股比例100張以下' ] + tdcc_df[ 'Share_Rating_Proportion_100001_200000']

tdcc_df['散戶持股比例400張以下'] = tdcc_df['散戶持股比例200張以下'] + \
                                 tdcc_df[ 'Share_Rating_Proportion_200001_400000']
    
tdcc_df['散戶持股比例600張以下'] = tdcc_df['散戶持股比例400張以下'] + \
                                 tdcc_df[ 'Share_Rating_Proportion_400001_600000']
    
tdcc_df['散戶持股比例800張以下'] = tdcc_df['散戶持股比例600張以下'] + \
                                 tdcc_df[ 'Share_Rating_Proportion_600001_800000']
    
tdcc_df['大戶持股比例400張以上'] = tdcc_df['Share_Rating_Proportion_400001_600000']  + \
tdcc_df['Share_Rating_Proportion_600001_800000']  + \
tdcc_df['Share_Rating_Proportion_800001_1000000'] + \
tdcc_df['Share_Rating_Proportion_Up_1000001']

tdcc_df['大戶持股比例600張以上'] = tdcc_df['大戶持股比例400張以上'] - tdcc_df['Share_Rating_Proportion_400001_600000']

tdcc_df['大戶持股比例800張以上'] = tdcc_df['大戶持股比例600張以上'] - tdcc_df['Share_Rating_Proportion_600001_800000']

tdcc_df['大戶持股比例1000張以上'] = tdcc_df['大戶持股比例800張以上'] - tdcc_df['Share_Rating_Proportion_800001_1000000']

In [14]:
# tdcc_df = tdcc_df[::-1]

fig = tls.make_subplots( rows=1, cols=1, shared_xaxes=False )
#                         subplot_titles=( '集中度', 'K線', '主力買賣超', '主力買賣超') )

fig.append_trace( {'x': tdcc_df['date'], 
                   'y': tdcc_df['大戶持股比例600張以上'], 
                   'type': 'scatter', 
                   'name': '大戶持股比例600張以上' }, 1, 1 )

fig.append_trace( {'x': tdcc_df['date'], 
                   'y': tdcc_df['大戶持股比例800張以上'], 
                   'type': 'scatter', 
                   'name': '大戶持股比例800張以上' }, 1, 1 )

fig.append_trace( {'x': tdcc_df['date'], 
                   'y': tdcc_df['大戶持股比例1000張以上'], 
                   'type': 'scatter', 
                   'name': '大戶持股比例1000張以上' }, 1, 1 )

fig.append_trace( {'x': tdcc_df['date'], 
                   'y': tdcc_df['散戶持股比例200張以下'], 
                   'type': 'scatter', 
                   'name': '散戶持股比例200張以下' }, 1, 1 )

fig.append_trace( {'x': tdcc_df['date'], 
                   'y': tdcc_df['散戶持股比例400張以下'], 
                   'type': 'scatter', 
                   'name': '散戶持股比例400張以下' }, 1, 1 )

fig.append_trace( {'x': tdcc_df['date'], 
                   'y': tdcc_df['散戶持股比例800張以下'], 
                   'type': 'scatter', 
                   'name': '散戶持股比例800張以下' }, 1, 1 )

# fig.append_trace( {'x': csv_df['日期'], 
#                    'y': csv_df['收盤'], 
#                    'type': 'scatter', 
#                    'name': '收盤' }, 2, 1 )

py.offline.iplot( fig, filename= '持股比例' )

This is the format of your plot grid:
[ (1,1) x1,y1 ]



In [15]:
# py.offline.plot( fig, filename = filename )